In [1]:
import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
import seaborn as sns
from fim import arules

In [2]:
# load the main file
fact_table = pd.read_csv('fact_table.csv')

# Lets Explore and Understand the Data


In [3]:
# Sales Trends Over Time + line 
# product items
# top customers
# sales by location
# Payment Method Preferences
# Time of Day Analysis

In [4]:
fact_table

,payment_key,coustomer_key,time_key,item_key,store_key,quantity,unit,unit_price,total_price
0,P026,C004510,T049189,I00177,S00307,1,ct,35.0,35.0
1,P022,C008967,T041209,I00248,S00595,1,rolls,26.0,26.0
2,P030,C007261,T03633,I00195,S00496,8,ct,12.5,100.0
3,P032,C007048,T084631,I00131,S0086,8,ct,14.0,112.0
4,P014,C006430,T071276,I00050,S00488,8,cans,8.0,64.0
...,...,...,...,...,...,...,...,...,...
999995,P010,C003726,T09247,I00063,S00502,5,cans,40.0,200.0
999996,P021,C002581,T023483,I00252,S00225,1,ct,11.0,11.0
999997,P035,C006133,T039372,I00119,S00555,7,ct,53.0,371.0
999998,P031,C008735,T097975,I00257,S00440,2,ct,21.0,42.0


In [5]:
#load the rest of the files 

In [6]:
item = pd.read_csv('item_dim.csv',  encoding='latin-1')

In [7]:
customer = pd.read_csv('customer_dim.csv', encoding='latin-1')

In [8]:
store = pd.read_csv('store_dim.csv', encoding='latin-1')

In [9]:
time_dim = pd.read_csv('time_dim.csv', encoding='latin-1')

In [10]:
trans = pd.read_csv('Trans_dim.csv', encoding='latin-1')

In [11]:
# and merge all tables
df = pd.merge(fact_table, time_dim, how='inner')

In [12]:
df = pd.merge(df, trans, how='inner')

In [13]:
df = pd.merge(df, store, how='inner')

In [14]:
df = pd.merge(df, item, how='inner')

In [15]:
df = pd.merge(df, customer, how='inner')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 0 to 999999
Data columns (total 28 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   payment_key    1000000 non-null  object 
 1   coustomer_key  1000000 non-null  object 
 2   time_key       1000000 non-null  object 
 3   item_key       1000000 non-null  object 
 4   store_key      1000000 non-null  object 
 5   quantity       1000000 non-null  int64  
 6   unit           996277 non-null   object 
 7   unit_price     1000000 non-null  float64
 8   total_price    1000000 non-null  float64
 9   date           1000000 non-null  object 
 10  hour           1000000 non-null  int64  
 11  day            1000000 non-null  int64  
 12  week           1000000 non-null  object 
 13  month          1000000 non-null  int64  
 14  quarter        1000000 non-null  object 
 15  year           1000000 non-null  int64  
 16  trans_type     1000000 non-null  object 
 17  bank_name

In [16]:
df.head()

,payment_key,coustomer_key,time_key,item_key,store_key,quantity,unit,unit_price,total_price,date,hour,day,week,month,quarter,year,trans_type,bank_name,division,district,upazila,item_name,desc,man_country,supplier,name,contact_no,nid
0,P026,C004510,T049189,I00177,S00307,1,ct,35.0,35.0,26-04-2016 17:13,17,26,4th Week,4,Q2,2016,card,Premier Bank Limited,DHAKA,JAMALPUR,ISLAMPUR,M&M Peanut Candy 1.7 oz,Food - Chocolate,Netherlands,Bolsius Boxmeer,priya,8801529462805,4543429591184
1,P001,C004510,T082988,I00062,S00428,11,cans,37.0,407.0,11-05-2018 07:18,7,11,2nd Week,5,Q2,2018,cash,None,CHITTAGONG,CHITTAGONG,ANOWARA,Red Bull 16oz,Beverage - Energy/Protein,Bangladesh,DENIMACH LTD,priya,8801529462805,4543429591184
2,P018,C004510,T074754,I00196,S00378,8,ct,16.0,128.0,29-11-2015 10:19,10,29,4th Week,11,Q4,2015,card,Mutual Trust Bank Limited,DHAKA,RAJBARI,KALUKHALI,Fiber One 90 Calorie Fudge Brownies,Food - Healthy,poland,CHROMADURLIN S.A.S,priya,8801529462805,4543429591184
3,P011,C004510,T06276,I00223,S00367,8,ct,13.5,108.0,11-10-2016 23:06,23,11,2nd Week,10,Q4,2016,card,Eastern Bank Limited,DHAKA,NETRAKONA,BARHATTA,Lance Variety Cracker Packs,Food - Snacks,Bangladesh,DENIMACH LTD,priya,8801529462805,4543429591184
4,P034,C004510,T08402,I00229,S00535,9,ct,13.0,117.0,17-10-2017 23:17,23,17,3rd Week,10,Q4,2017,card,United Commercial Bank Ltd,BARISAL,BHOLA,LALMOHAN,Kellogg's Cereals Assortment Pack,Food - Sweets,Germany,Friedola 1888 GmbH,priya,8801529462805,4543429591184


In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df[df['unit'].isnull()].sort_values('item_key')

In [ ]:
item[item['item_key'] == 'I00158']

In [ ]:
df['unit'].fillna('Frito Bold Flavors Variety', inplace=True)

In [ ]:
df.info()

# RFM Analysis

In [ ]:
# create time_difference to capture part of data below
df["time_difference"] = df['date'].max() - df["date"]

In [ ]:
#last two years
df["last_two_years"] = df["time_difference"] <= timedelta(days=2*365)

In [ ]:
df_RFM = df[df['last_two_years']==True].copy()

In [ ]:
df_RFM['Recency'] = (df_RFM['date'].max() - df_RFM['date']).dt.days


In [ ]:
df_RFM.head()

### Using group by for each customer we need we create three now columns as following

 -Recency: for each customer snapshot date – latest purchase difference in days
 
 -Frequency: for each customer count of how many invoices are issued for him
 
 -Monetary Value: for each customer sum of amount spent

In [ ]:
df_RFM = df_RFM.groupby(['coustomer_key']).agg({'Recency': np.min,
                                           'date':pd.Series.nunique,
                                           'total_price':np.sum}).reset_index()

df_RFM

In [ ]:
#Rename columns
df_RFM.rename(columns={'Recency':'Recency','date':'Frequency','total_price':'Monetary'},inplace= True)

df_RFM

### We have now calculated our values. But those aggregations are absolut and hard to explain as is. It is better to normalize our scores replacing those values with relative once with respect to the distribution

### Pandas qcut splits numeric columns into quartiles.

### For Recency and Frequency columns, the higher values (top quartile) are assigned with indicator 4 and lowest to 1. Higher values imply that customers have spent more money to our products or bought more often and the 1-4 score is assigned accordingly

### The opposite logic is applied for recency as higher values mean that many days have passed since last purchase. Thus, customers with high values (top quartile) take score 1 and those with the smallest values 4.

In [ ]:
# Create Scores RFM scores based on quantiles of distribution

#Date from customer's last purchase.The nearest date gets 4 and the furthest date gets 1.
df_RFM["recency_score"] = pd.qcut(df_RFM['Recency'].rank(method="first"),
                                  4,
                                  labels=[4, 3, 2, 1])

# Total number of purchases.The least frequency gets 1 and the maximum frequency gets 4.
df_RFM["frequency_score"] = pd.qcut(df_RFM["Frequency"].rank(method="first"),
                                    4,
                                    labels=[1, 2, 3, 4])

#Total spend by the customer.The least money gets 1, the most money gets 4.
df_RFM["monetary_score"] = pd.qcut(df_RFM["Monetary"].rank(method="first"),
                                   4,
                                   labels=[1, 2, 3, 4])

df_RFM.head()

## Calculate total RFM score for each customer


In [ ]:
df_RFM["RFM_Segment"] = df_RFM["recency_score"].astype(str) + df_RFM[
    "frequency_score"].astype(str) + df_RFM["monetary_score"].astype(str)

df_RFM['RFM_Score'] = df_RFM[[
    'recency_score', 'frequency_score', 'monetary_score'
]].sum(axis=1)
df_RFM.head()

### Split customers to segments and give indicative names (labeling appoaches may vary for this step)


In [ ]:
segt_map = {
    r'[3-4][3-4]4': 'VIP',
    r'[2-3-4][1-2-3-4]4': 'Top Recent',
    r'1[1-2-3-4]4': 'Top at Risk ',

    
    
    r'[3-4][3-4]3': 'High Promising',
    r'[2-3-4][1-2]3': 'High New',
    r'2[3-4]3': 'High Loyal',

    
    
    r'[3-4][3-4]2': 'Medium Potential',
    r'[2-3-4][1-2]2': 'Medium New',
    r'2[3-4]2': 'Medium Loyal',

    
    
    r'4[1-2-3-4]1': 'Low New',
    r'[2-3][1-2-3-4]1': 'Low Loyal',
    
    r'1[1-2-3-4][1-2-3]': 'Need Activation'
}
df_RFM['Segment_labels'] = df_RFM['RFM_Segment']
df_RFM['Segment_labels'] = df_RFM['Segment_labels'].replace(segt_map, regex=True)
df_RFM.head()

In [ ]:
seg_pareto = df_RFM.groupby(["Segment_labels"]).agg({'Monetary': np.sum,
                 
                                                           "coustomer_key": pd.Series.nunique}).reset_index()

seg_pareto

In [ ]:
seg_pareto["Monetary%"] = seg_pareto["Monetary"]/seg_pareto["Monetary"].sum()
seg_pareto = seg_pareto.sort_values(by=['Monetary%'], ascending=False)
seg_pareto["CumulativePercentage"] = (seg_pareto["Monetary"].cumsum()/ 
                                      seg_pareto["Monetary"].sum()*100).round(2)
seg_pareto["CumulativeSum"] = (seg_pareto["coustomer_key"].cumsum()/ 
                                      seg_pareto["coustomer_key"].sum()*100).round(2)

seg_pareto

# Market Basket Analysis


## Association Rules


In [ ]:
df.info()

## 1 - Produce an Association Rules Report which will show the below measures: 
### support itemset absolute, support itemset relative pct, confidence pct and lift

In [ ]:
# inputs
supp = 5  # minimum support of an assoc. rule (default: 10)
conf = 30  # minimum confidence of an assoc. rule (default: 80%)
zmin = 2
zmax = 3
report = 'aSCl'

In [ ]:
report_colnames = {
    'a': 'support_itemset_absolute',
    's': 'support_itemset_relative',
    'S': 'support_itemset_relative_pct',
    'b': 'support_bodyset_absolute',
    'x': 'support_bodyset_relative',
    'X': 'support_bodyset_relative_pct',
    'h': 'support_headitem_absolute',
    'y': 'support_headitem_relative',
    'Y': 'support_headitem_relative_pct',
    'c': 'confidence',
    'C': 'confidence_pct',
    'l': 'lift',
    'L': 'lift_pct',
    'e': 'evaluation',
    'E': 'evaluation_pct',
    'Q': 'support of the empty set (total number of transactions)'
    }

In [ ]:
cust_prod=df.groupby('coustomer_key')['item_name'].apply(list)
cust_prod

In [ ]:
# Create list of lists to pass it into PyFim

cust_prod_list=cust_prod.to_list()
cust_prod_list

In [ ]:
# Run apriori algorithm to creeate associations
apriori = arules(cust_prod_list, supp=supp, conf=conf, report=report,eval='lift', zmin = 2,zmax = 2)

In [ ]:
#Uncomment and run in case you want to see parameters of association rules
#??arules 


#Creat dataframe wiht columns based on the report variables and sorted based on the "Associations_Sorted_based_on"
colnames = ['Cons_Product', 'antecedent'] + [report_colnames.get(k, k) for k in list(report)]
df_rules = pd.DataFrame(apriori, columns=colnames)
df_rules = df_rules.sort_values(report_colnames["a"], ascending=False)

#Change order of columns antecedent and consequent
df_rules = df_rules[['antecedent', 'Cons_Product']+ [report_colnames.get(k, k) for k in list(report)]]


#Print the numbers of rules created
print(df_rules.shape)

#Print top 10 and last 10 rules (the last 10 will be shown)
df_rules

## 2 - Find the Top5 Association in terms of Support Absolute, confidence and lift and provide an interpretation of your results for these product relations

In [ ]:
df_rules = df_rules.sort_values(by='lift', ascending=False)
df_rules.head(30)

In [ ]:
df_rules = df_rules.sort_values(by='confidence_pct', ascending=False)
df_rules.head(30)

## 3 - Is there product cannibalization between products? If yes, provide for which products. If not, support your statement.

In [ ]:
df_rules = df_rules.sort_values(by='lift', ascending=True)
df_rules.head(50)